In [2]:
import random
import os
import glob
import csv
import shutil
import math
import json
from itertools import combinations, product, islice

import pandas as pd
import numpy as np
from multiprocessing import Pool, Lock, cpu_count
from concurrent.futures import ThreadPoolExecutor
from scipy import stats

In [64]:
# configure data path
current_path = os.path.dirname(os.path.abspath("__file__"))
data_path = os.path.join(current_path, "..", "data")

data_path = os.path.normpath(data_path)
print(data_path)

/home/dk/Desktop/space_weather_econ/data


In [65]:
# read the business and economic data at zcta and state levels

data_zcta = pd.read_csv(os.path.join(data_path, "df_zcta.csv"))
data_state = pd.read_csv(os.path.join(data_path, "df_rto_stats.csv"))

In [66]:
# state levels
data_state.groupby("REGIONS")[["POP20BG"]].sum().reset_index()

,REGIONS,POP20BG
0,CAISO,36592680
1,ERCOT,25771826
2,FRCC,20332393
3,ISONE,14155945
4,MISO,44732745
5,NYISO,22038595
6,NorthernGridConnected,16560671
7,NorthernGridUnconnected,4327744
8,NotOrder1000,2551988
9,PJM,66127547


### Business Count at the Regions and their Estimated GDP
In our calculation, we assume uniform every person/business contribute equally to State GDP.

In [67]:
# zcta
df_business = data_zcta.groupby("REGIONS")[["EMP", "EST", "POP20", "ZCTAGDP"]].sum().reset_index()
df_business.rename(columns={"ZCTAGDP": "GDPREGION"}, inplace=True)

# add daily GDP
df_business["daily_GDP"] = df_business.GDPREGION.map(lambda x: x*1/365)
print(df_business.GDPREGION.sum())

df_business.head(5)

21309469.75564568


,REGIONS,EMP,EST,POP20,GDPREGION,daily_GDP
0,CAISO,12953016,930647,36669676,2.926565e+06,8017.987086
1,ERCOT,9363417,571573,25756383,1.699788e+06,4656.952155
2,FRCC,6979790,585058,20269759,1.146682e+06,3141.594334
3,ISONE,5779762,366502,14160438,1.072352e+06,2937.949888
4,MISO,16998586,1051362,44656227,2.563673e+06,7023.760705


### Loss Estimation by Randomly Assigning Percentage of Businesses and Population without Power

In [68]:
# simplifiied code for GDP Loss estimation
def compute_GDP_loss(df, baseline_failures: list, time: int=24):
    """
    Computes the GDP loss due to a power failure.

    Parameters:
    baseline_failures (list): List of percentages of the population affected by the power failure.
    time (int): The duration of the power failure in hours.

    Returns:
    DataFrame: A DataFrame with regions and their corresponding GDP losses under the scenario
    for all population without power.
    """
    scenarios = []
    for f in baseline_failures:
        # scenario string
        scenario_str = f"scenario_{int(f * 100)}f_{time}h"
        # add column to scenarios list
        scenarios.append(scenario_str)

        # calculating GDP string
        df[scenario_str] = df["daily_GDP"].map(lambda x: x * f * (time / 24))

    scenarios.extend(["REGIONS"])
    scenarios.reverse()
    
    return df[scenarios]


# List of baseline failure rates
baseline_failures = [0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 0.75, 0.975, 1]

df_est_loss = compute_GDP_loss(df_business, baseline_failures)

In [69]:
df_est_loss.head(10)

,REGIONS,scenario_100f_24h,scenario_97f_24h,scenario_75f_24h,scenario_50f_24h,scenario_25f_24h,scenario_10f_24h,scenario_5f_24h,scenario_2f_24h,scenario_1f_24h
0,CAISO,8017.987086,7817.537409,6013.490314,4008.993543,2004.496771,801.798709,400.899354,200.449677,80.179871
1,ERCOT,4656.952155,4540.528351,3492.714116,2328.476078,1164.238039,465.695216,232.847608,116.423804,46.569522
2,FRCC,3141.594334,3063.054476,2356.195751,1570.797167,785.398584,314.159433,157.079717,78.539858,31.415943
3,ISONE,2937.949888,2864.501141,2203.462416,1468.974944,734.487472,293.794989,146.897494,73.448747,29.379499
4,MISO,7023.760705,6848.166687,5267.820529,3511.880352,1755.940176,702.376070,351.188035,175.594018,70.237607
5,NYISO,5167.930314,5038.732056,3875.947736,2583.965157,1291.982579,516.793031,258.396516,129.198258,51.679303
6,NorthernGridConnected,2930.427306,2857.166623,2197.820480,1465.213653,732.606827,293.042731,146.521365,73.260683,29.304273
7,NorthernGridUnconnected,934.480353,911.118344,700.860265,467.240176,233.620088,93.448035,46.724018,23.362009,9.344804
8,NotOrder1000,545.395416,531.760531,409.046562,272.697708,136.348854,54.539542,27.269771,13.634885,5.453954
9,PJM,11555.533987,11266.645637,8666.650490,5777.766993,2888.883497,1155.553399,577.776699,288.888350,115.555340


In [70]:
# sum of daily economic losses in millions for the given scenarios
cols = list(df_est_loss.columns)[1:]

df_est_loss[cols].sum(axis=0)

scenario_100f_24h    58382.108920
scenario_97f_24h     56922.556197
scenario_75f_24h     43786.581690
scenario_50f_24h     29191.054460
scenario_25f_24h     14595.527230
scenario_10f_24h      5838.210892
scenario_5f_24h       2919.105446
scenario_2f_24h       1459.552723
scenario_1f_24h        583.821089
dtype: float64

### Uncertainty Quantification

In the previous model, we assume equal percentages of populations within the regions to be affected without power. 
Some regions are more vulnerable or least resilient to space weather.

In [100]:
def rand_without_power(df, time=24, num_simulations_list=[999], max_failure=1):
    
    # simulation columns
    sim_cols = []
    
    # loop over the number of permutations given
    for num_simulations in num_simulations_list:
        
        # running Monte Carlo simulations
        simulation_results = []

        for _ in range(num_simulations):
            
            # randomly selecting failure rate of a region (0-1)
            failure_probability = np.random.rand() * max_failure

            # calculating the GDP loss for each region with the randomly selected probability
            result = df['daily_GDP'].map(lambda x: x * failure_probability * (time / 24))
            simulation_results.append(result)

        # calculate the average result from all simulations
        avg_simulation_result = pd.concat(simulation_results, axis=1).mean(axis=1)
        sim_str = f"avg_loss_{num_simulations}_sim"
        df[sim_str] = avg_simulation_result
        
        sim_cols.append(sim_str)
        
    sim_cols.extend(["REGIONS"])
    sim_cols.reverse()
    
    return df[sim_cols]


# Example Usage
simulations_list = [999, 9999, 99999]

# taking the maximum number of people in a population without power is 50%
df_simulations = rand_without_power(df_business, time=24, num_simulations_list=simulations_list, max_failure=1)

df_simulations.head(5)

,REGIONS,avg_loss_99999_sim,avg_loss_9999_sim,avg_loss_999_sim
0,CAISO,4003.327235,4021.590764,3928.893238
1,ERCOT,2325.185012,2335.792709,2281.952769
2,FRCC,1568.577004,1575.732989,1539.412399
3,ISONE,1466.898696,1473.590817,1439.624600
4,MISO,3506.916664,3522.915525,3441.712445


In [102]:
# sum of the simulations
cols = list(df_simulations.columns)
cols.remove("REGIONS")

df_simulations[cols].sum(axis=0)

avg_loss_99999_sim    29149.795849
avg_loss_9999_sim     29282.779769
avg_loss_999_sim      28607.812719
dtype: float64

### Considering Each Region Differ in Population without Power

In [103]:
def rand_without_power_ind(df, time=24, num_simulations_list=[999], max_failure=1):
    
    "Make failure rate in the region independent of each other"
    
    # simulation columns
    sim_cols = []
    
    # loop over the number of permutations given
    for num_simulations in num_simulations_list:
        
        # running Monte Carlo simulations
        simulation_results = []

        for _ in range(num_simulations):
            
            # randomly selecting failure rate of a region (0-1)
            failure_probabilities = np.random.rand() * max_failure # add maximum failure rate
            gdp_losses = df['daily_GDP'] * failure_probabilities * (time/24) # include period without power

            simulation_results.append(gdp_losses)

        # calculate the average result from all simulations
        avg_simulation_result = pd.concat(simulation_results, axis=1).mean(axis=1)
        sim_str = f"avg_loss_{num_simulations}_sim"
        df[sim_str] = avg_simulation_result
        
        sim_cols.append(sim_str)
        
    sim_cols.extend(["REGIONS"])
    sim_cols.reverse()
    
    return df[sim_cols]


# Example Usage
simulations_list = [999, 9999, 99999]

# taking the maximum number of people in a population without power is 50%
df_simulations = rand_without_power(df_business, time=24, num_simulations_list=simulations_list, max_failure=1)

df_simulations.head(5)

,REGIONS,avg_loss_99999_sim,avg_loss_9999_sim,avg_loss_999_sim
0,CAISO,4005.007635,4002.104147,3980.545546
1,ERCOT,2326.161010,2324.474626,2311.953107
2,FRCC,1569.235415,1568.097775,1559.650720
3,ISONE,1467.514428,1466.450532,1458.551032
4,MISO,3508.388695,3505.845238,3486.959893


In [104]:
# sum of the simulations
cols = list(df_simulations.columns)
cols.remove("REGIONS")

df_simulations[cols].sum(axis=0)

avg_loss_99999_sim    29162.031502
avg_loss_9999_sim     29140.890063
avg_loss_999_sim      28983.913434
dtype: float64